In [ ]:
from libs.plugin_converter.semantic_kernel_v0_to_openai_function import generate_callables, generate_definitions, make_context
from libs.plugins.plugin_capital import PluginCapital
import libs.plugin_converter.semantic_kernel_v0_to_openai_function as semantic_kernel_v0_to_openai_function
import libs.plugin_converter.openai_function_to_tool as openai_function_to_tool

import os
from libs.utils.connector_llm import *
from libs.plugins.plugin_capital import PluginCapital
import libs.plugin_converter.semantic_kernel_v0_to_openai_function as semantic_kernel_v0_to_openai_function
import libs.plugin_converter.openai_function_to_tool as openai_function_to_tool
from libs.plugin_orchestrator.implementation_tool import OrchestratorWithTool
from libs.plugin_orchestrator.implementation_bare import OrchestratorBare

import dotenv
dotenv.load_dotenv()

In [ ]:
llm = factory_create_connector_llm(
    provider='llama_cpp',
    modelname='not-needed',
    version='not-needed',
    credentials=CredentialsOpenAI(
        base_url='http://llm-server:8080/v1',
        api_key='not-needed',
    )
)

#llm = factory_create_connector_llm(
#    provider='azure_openai',
#    modelname=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME_CHAT"),
#    version=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME_CHAT").split('-')[-1],
#    credentials=CredentialsOpenAI(
#        base_url=os.getenv("AZURE_OPENAI_ENDPOINT"),
#        api_key=os.getenv("AZURE_OPENAI_KEY"),
#    ),
#    hyperparameters={'tool_choice': 'none'}
#)

orchestrator = OrchestratorBare(
    connection = llm,
    tool_definitions = openai_function_to_tool.generate_definitions(semantic_kernel_v0_to_openai_function.generate_definitions([(PluginCapital(), "PluginCapital")])),
    tool_callables = generate_callables([(PluginCapital(), "PluginCapital")]),
    token_limit_input = 1024,
    token_limit_output = None,
    max_steps_recommended = 2,
    max_steps_allowed = 3,
    prompt_app_system='You are an AI assistant whose goal is to answer the user question.',
    prompt_app_user='What is the capital of france?',
)

r = await orchestrator.run()
print(r)
print('\n\n' + '-'*30 + '\n\n')
assert type(r.answer) == str
assert len(r.answer) > 4
print(r.answer)

In [ ]:
#############################################
# Dataset generation
orchestrator._full_message_history_for_debugging
# This is the expected chat-over-text format:
# <|system|> You are a chatbot who can help code!</s> <|user|> Write me a function to calculate the first 10 digits of the fibonacci sequence in Python and print it out to the CLI.</s> <|assistant|>
# Before finetunning, we have to convert
# this format is called "Zephyr", because it is used in the model HuggingFaceH4/zephyr-7b-alpha